In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\end_to_end_flight_price_prediction\\FLIGHT_PRICE_PREDICTION'

In [10]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    max_features: str
    min_samples_split: int
    n_estimators: int
    target_column: str

In [11]:
from Flight_Price.constants import *
from Flight_Price.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Create artifact root directory
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params["RandomForestRegressor"]
        schema = self.schema["TARGET_COLUMN"]

        # Create directories for model trainer
        create_directories([config["root_dir"]])

        # Initialize ModelTrainerConfig
        model_trainer_config = ModelTrainerConfig(
            root_dir=config["root_dir"],
            train_data_path=config["train_data_path"],
            test_data_path=config["test_data_path"],
            model_name=config["model_name"],
            max_depth=params["max_depth"],
            max_features=params["max_features"],
            min_samples_split=params["min_samples_split"],
            n_estimators=params["n_estimators"],
            target_column=schema["name"]
        )

        return model_trainer_config


In [13]:
import pandas as pd
import os
from Flight_Price.logging import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        ml_model=RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_split=10,n_estimators=400)
        ml_model.fit(train_x,train_y)

        joblib.dump(ml_model, os.path.join(self.config.root_dir, self.config.model_name))

In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-02 19:04:50,058: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-02 19:04:50,061: INFO: common: yaml file: Params.yaml loaded successfully]
[2024-12-02 19:04:50,067: INFO: common: yaml file: Schema.yaml loaded successfully]
[2024-12-02 19:04:50,070: INFO: common: created directory at: artifacts]
[2024-12-02 19:04:50,071: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\Lenovo\anaconda3\envs\mlflight\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
